# Introduction to Linear Programming with Scipy

## Try me

{{ badges }}

## Requirements
### Install in your environment
#### Scipy Installation
The simplest way to install SciPy in your environment is using [pip](https://pypi.org/project/pip/). If you have installed
Python and pip in your environment, just open a terminal and try:

```
pip install scipy
```
#### Conda Installation
If you use Conda, open a Conda Terminal and try: 

```
conda install scipy
```

#### Google Colabs installation
Scipy is already installed in Google Colabs, no installation required!

#### Binder installation
Run the following code cell to try this notebook in Binder:


In [1]:
!pip install pandas
!pip install numpy
!pip install scipy


## Linear Optimisation with Scipy
In this tutorial, we will learn to model and solve Linear Programming Problems using the Python open source scientific library  [Scipy](https://scipy.org/). SciPy is an awesome library extensively used for scientific and technical computing. It is built on top of NumPy and provides a wide range of functionality including optimization, signal processing, interpolation, and more. It also contains modules for linear algebra, optimization, and integration. SciPy is widely used in the scientific and engineering communities and is a powerful tool for data analysis and visualization.

In this tutorial, we will learn how to use SciPy to model and solve CLP problems. Just as with the previous tutorial, to guide this example, we will use a simple CLP formulated in class:

maximise $z = 300x + 250y$

Subject to:

$$2x + y \leq 40$$


$$x + 3y \leq 45$$


$$x \leq 12$$


In [2]:
# Let´s start importing the linprog function of the optimize package of SciPy
from scipy.optimize import linprog
# We are going to use panda to display the results as tables using Panda
import pandas as pd
#And we will use numpy to perform array operations
import numpy as np
#We will use display and Markdown to format the output of code cells as Markdown
from IPython.display import display, Markdown

### Problem function linprog
#### Transforming the problem model
The function ```linprog``` of the Scipy package can be used to solve continuous linear programming problems expressed in the form:

$\min z = c^T*x$

s.t.
$A_{ub}*x \leq b_{ub}$

$A_{uc}*x = b_{uc}$

$l \leq x \leq u$

Where $x$ is a (column) vector with the decision variables, $c$ is a (column) vector with the objective function coefficients, $z$ is the objective variable (scalar), $A_{ub}$ is a matrix with the LHS coefficients of the constraints of type *less or equal*, and $b_ub$ is a vector that contains the corresponding RHS coefficients of the same constraints, and finally, $A_{uc}$ is a matrix with the LHS coeffients of type *equal*, and $b_ub$ is a vector that contains the corresponding RHS coefficients.

This means that we need to convert our problem to comply with the format expected by ```linprog```. In our example, we can convert the objective function as:


$\min z* = - z = -300*x - 250y = [-300, -250]^T*[x, y]$

That is, since our objective function is of type *maximize*, we use the equivalent minimization problem and the solution will be the negative of our original objective variable. As for the constraint, we need to express all the constraint (except the bounds) as of type *less_or_equal*. In our case, we do not need to apply any transformation, and our matrix $A_{ub}$ becomes:


$A_{ub} = \begin{bmatrix}
2 & 1\\
1 & 3
\end{bmatrix}$

Note that the first row correspond to the coefficients of the first constraint, and the coefficients in the second row, to the second constraint, therefore, the vector $b_{ub}$ is:

$b_{ub} = [40, 45]^T$

Now, $l$ is going to contain the minimum values of the decision variables, or **lower bound**. Since both variables need to be non-negative:

$l = [0, 0]$

and finally, $u$ is going to contain the maximum values or **upper bound**, since x cannot be higher than 12, the upper bounds are:


$u = [12, \infty]$

#### linprog function
The documentation of linprog can be found [here](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.linprog.html#scipy.optimize.linprog), but in short, it takes the following arguments:

**linprog** ***(c, A_ub=None, b_ub=None, A_eq=None, b_eq=None, bounds=None, method='highs')***

- **c**: 1-D array containing the coefficients of the linear objective function to be minimized.

- **A_ub**: 2-D array (optional). The inequality constraint matrix. Each row of A_ub specifies the coefficients of a linear inequality constraint on x.

- **b_ub**: 1-D array (optional). The inequality constraint vector. Each element represents an upper bound on the corresponding value of A_ub @ x.

- **A_eq**: 2-D array (optional). The equality constraint matrix. Each row of A_eq specifies the coefficients of a linear equality constraint on x.

- **b_eq**: 1-D array (optional). The equality constraint vector. Each element of A_eq @ x must equal the corresponding element of b_eq.

- **bounds**: sequence: (optional). A sequence of (min, max) pairs for each element in x, defining the minimum and maximum values of that decision variable. Use None to indicate that there is no bound. By default, bounds are (0, None) (all decision variables are non-negative). If a single tuple (min, max) is provided, then min and max will serve as bounds for all decision variables.

- **method**: string (optional). This is the method-specific documentation for ‘highs’, which chooses automatically between ‘highs-ds’ and ‘highs-ipm’. ‘interior-point’ (default), ‘revised simplex’, and ‘simplex’ (legacy) are also available.

In [3]:
# objective coefficient vector
c = [-300, -250]
# Inequality constraints LHS matrix
A = [[2, 1], [1, 3]]
# Inequality constraints RHS
b = [40, 45]

#Bounds for x
x_bounds = (0, 12)
# Bounds for y
y_bounds = (0, None)

res = linprog(c, A_ub=A, b_ub=b, bounds=[x_bounds, y_bounds])
print(f"result of the objective function is: {res.fun}")
print(f"result of the decision variables is: {res.x}")

print(res.message)


result of the objective function is: -6350.0
result of the decision variables is: [12. 11.]
Optimization terminated successfully. (HiGHS Status 7: Optimal)


Now, let us display the solution in a nice table using Pandas. We are going to first display the solution value using markdown and then we will use Pandas to create a table with the results.

In [16]:
var_df = pd.DataFrame(index=['x', 'y'])
var_df['solution'] = res.x
var_df['coefficients'] = c
display(var_df)

,solution,coefficients
x,12.0,-300
y,11.0,-250


And now another table with the constraints:

In [17]:
con_df = pd.DataFrame(index=['Man hours', 'Machine time', 'Marketing'])
con_df['RHS'] = b.append(x_bounds[1])
con_df['slacks'] = np.append(res.slack, res.upper.residual[0])
con_df['shadow_prices'] = np.append(res.ineqlin.marginals, res.upper.marginals[0])
display(con_df)

,RHS,slacks,shadow_prices
Man hours,None,5.0,-0.000000
Machine time,None,0.0,-83.333333
Marketing,None,0.0,-216.666667
